In [19]:
import pandas as pd
import pycountry
import time

def merge_movie_data():
    """Merge csv files."""
    start_time = time.time()

    # Load and filter title basics

    columns_to_load = ["tconst","originalTitle","titleType","startYear","runtimeMinutes","genres"]
    title_basics = pd.read_csv('assets/title.basics.csv', usecols=columns_to_load, engine='pyarrow')

    # Filter the DataFrame based on the `titleType` column
    title_basics_filtered = title_basics.query('titleType in ["movie", "tvSeries"]')

    print(f"time to load title basics: {time.time() - start_time:.2f} s")

    # Load and process IMDb movies
    imdb_movies = pd.read_csv('assets/imdb_movies.csv', usecols=['orig_title', 'overview', 'budget_x', 'revenue', 'country', 'genre', 'date_x'])
    imdb_movies['year_x'] = pd.to_datetime(imdb_movies['date_x'], errors='coerce').dt.year.astype(str)


    # Merge IMDb movies with filtered title basics
    merged_data = pd.merge(
        imdb_movies, title_basics_filtered,
        left_on=['orig_title', 'year_x'],
        right_on=['originalTitle', 'startYear'],
        how='left'
    )

    # Load title ratings and merge with existing data
    title_ratings = pd.read_csv('assets/title.ratings.csv')
    merged_data = pd.merge(
        merged_data, title_ratings,
        on='tconst',
        how='left'
    )

    # Remove duplicates based on 'tconst'
    merged_data = merged_data.drop_duplicates(subset='tconst')

    merged_data['year_x'] = pd.to_numeric(merged_data['year_x'], errors='coerce')

    # Example: Convert ISO-2 column to ISO-3
    merged_data['country_name'] = merged_data['country'].apply(iso2_to_country_name)

    # Drop rows where conversion failed (invalid ISO-2 codes)
    merged_data = merged_data.dropna(subset=['country_name'])
    merged_data = merged_data.dropna(subset=['titleType'])


    # Clean the genre column by splitting comma-separated genres and trimming whitespace
    merged_data['genre'] = merged_data['genre'].apply(
        lambda x: [genre.strip() for genre in x.split(',')] if isinstance(x, str) else (x if isinstance(x, list) else [])
    )

    # Drop rows with empty genres, if necessary
    merged_data = merged_data[merged_data['genre'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

    print(f"time to run all: {time.time() - start_time:.2f} s")

    return merged_data


def iso2_to_country_name(iso2_code):
    try:
        # Convert ISO-2 to full country name using pycountry
        return pycountry.countries.get(alpha_2=iso2_code).name
    except AttributeError:
        return None  # Return None if the code is invalid or not found


# Example usage
if __name__ == "__main__":
    merged_data = merge_movie_data()



time to load title basics: 5.41 s
time to run all: 6.39 s


In [20]:
display(merged_data)

,date_x,genre,overview,orig_title,budget_x,revenue,country,year_x,tconst,originalTitle,titleType,startYear,runtimeMinutes,genres,averageRating,numVotes,country_name
0,03/02/2023,"[Drama, Action]","After dominating the boxing world, Adonis Cree...",Creed III,75000000.0,2.716167e+08,AU,2023,tt11145118,Creed III,movie,2023,116,"Action,Drama,Sport",6.7,98374.0,Australia
1,12/15/2022,"[Science Fiction, Adventure, Action]",Set more than a decade after the events of the...,Avatar: The Way of Water,460000000.0,2.316795e+09,AU,2022,tt1630029,Avatar: The Way of Water,movie,2022,192,"Action,Adventure,Fantasy",7.5,523442.0,Australia
2,04/05/2023,"[Animation, Adventure, Family, Fantasy, Comedy]","While working underground to fix a water main,...",The Super Mario Bros. Movie,100000000.0,7.244590e+08,AU,2023,tt6718170,The Super Mario Bros. Movie,movie,2023,92,"Adventure,Animation,Comedy",7.0,258549.0,Australia
4,03/17/2023,[Action],Good-hearted teenager William always lived in ...,Supercell,77000000.0,3.409420e+08,US,2023,tt10559102,Supercell,movie,2023,100,"Action,Adventure,Drama",4.4,3555.0,United States
5,02/23/2023,"[Thriller, Comedy, Crime]","Inspired by a true story, an oddball group of ...",Cocaine Bear,35000000.0,8.000000e+07,AU,2023,tt14209916,Cocaine Bear,movie,2023,95,"Comedy,Crime,Thriller",5.9,116631.0,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10426,07/10/2008,"[Comedy, Romance]",Born in America and raised in an Indian ashram...,The Love Guru,62000000.0,4.015902e+07,AU,2008,tt0811138,The Love Guru,movie,2008,87,"Comedy,Romance,Sport",3.8,55695.0,Australia
10428,10/07/1938,"[Mystery, Thriller, Comedy]",On a train headed for England a group of trave...,The Lady Vanishes,132000000.0,5.637564e+08,GB,1938,tt0030341,The Lady Vanishes,movie,1938,96,"Mystery,Thriller",7.7,58711.0,United Kingdom
10429,12/28/2016,[Drama],"In 1979 Santa Barbara, California, Dorothea Fi...",20th Century Women,7000000.0,9.353729e+06,US,2016,tt4385888,20th Century Women,movie,2016,119,"Comedy,Drama",7.3,50706.0,United States
10430,08/24/1990,[Action],When DEA agents are taken captive by a ruthles...,Delta Force 2: The Colombian Connection,9145817.8,6.698361e+06,US,1990,tt0099399,Delta Force 2: The Colombian Connection,movie,1990,111,"Action,Adventure,Crime",4.9,8426.0,United States


In [6]:
df = pd.read_csv('./assets/merged_data.csv')


unique_title_types = df['country'].unique()
print(unique_title_types)





['AU' 'US' 'GB' 'CL' 'DE' 'ES' 'FR' 'MX' 'GT' 'BR' 'ZA' 'NL' 'FI' 'ID'
 'DK' 'JP' 'IE' 'BE' 'AR' 'PH' 'TR' 'IT' 'CA' 'CH' 'CN' 'PE' 'SG' 'PL'
 'IN' 'GR' 'RU' 'TW' 'MY' 'MU' 'NO' 'UA' 'HK' 'SE' 'CO']


In [4]:
df = pd.read_csv('./assets/merged_data.csv')

display(df['tconst'].unique())



,orig_title,overview,budget_x,revenue,country,genre,date_x,year_x,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,Creed III,"After dominating the boxing world, Adonis Cree...",75000000.0,2.716167e+08,AU,"Drama, Action",03/02/2023,2023,tt11145118,movie,Creed III,Creed III,0,2023,\N,116,"Action,Drama,Sport",6.7,98374
1,Avatar: The Way of Water,Set more than a decade after the events of the...,460000000.0,2.316795e+09,AU,"Science Fiction, Adventure, Action",12/15/2022,2022,tt1630029,movie,Avatar: The Way of Water,Avatar: The Way of Water,0,2022,\N,192,"Action,Adventure,Fantasy",7.5,523442
2,The Super Mario Bros. Movie,"While working underground to fix a water main,...",100000000.0,7.244590e+08,AU,"Animation, Adventure, Family, Fantasy, Comedy",04/05/2023,2023,tt6718170,movie,The Super Mario Bros. Movie,The Super Mario Bros. Movie,0,2023,\N,92,"Adventure,Animation,Comedy",7.0,258549
3,Supercell,Good-hearted teenager William always lived in ...,77000000.0,3.409420e+08,US,Action,03/17/2023,2023,tt10559102,movie,Supercell,Supercell,0,2023,\N,100,"Action,Adventure,Drama",4.4,3555
4,Cocaine Bear,"Inspired by a true story, an oddball group of ...",35000000.0,8.000000e+07,AU,"Thriller, Comedy, Crime",02/23/2023,2023,tt14209916,movie,Cocaine Bear,Cocaine Bear,0,2023,\N,95,"Comedy,Crime,Thriller",5.9,116631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5086,The Love Guru,Born in America and raised in an Indian ashram...,62000000.0,4.015902e+07,AU,"Comedy, Romance",07/10/2008,2008,tt0811138,movie,The Love Guru,The Love Guru,0,2008,\N,87,"Comedy,Romance,Sport",3.8,55695
5087,The Lady Vanishes,On a train headed for England a group of trave...,132000000.0,5.637564e+08,GB,"Mystery, Thriller, Comedy",10/07/1938,1938,tt0030341,movie,The Lady Vanishes,The Lady Vanishes,0,1938,\N,96,"Mystery,Thriller",7.7,58711
5088,20th Century Women,"In 1979 Santa Barbara, California, Dorothea Fi...",7000000.0,9.353729e+06,US,Drama,12/28/2016,2016,tt4385888,movie,20th Century Women,20th Century Women,0,2016,\N,119,"Comedy,Drama",7.3,50706
5089,Delta Force 2: The Colombian Connection,When DEA agents are taken captive by a ruthles...,9145817.8,6.698361e+06,US,Action,08/24/1990,1990,tt0099399,movie,Delta Force 2: The Colombian Connection,Delta Force 2: The Colombian Connection,0,1990,\N,111,"Action,Adventure,Crime",4.9,8426


5091


In [30]:
duplicate_tconsts = df[df.duplicated('tconst', keep=False)]
display(duplicate_tconsts)



,orig_title,overview,budget_x,revenue,country,genre,date_x,year_x,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
64,Scream,Twenty-five years after a streak of brutal mur...,24000000.0,1.388748e+08,AU,"Horror, Mystery, Thriller",01/13/2022,2022,tt11245972,movie,Scream,Scream,0,2022,\N,114,"Horror,Mystery,Thriller",6.3,169978
65,Scream,Twenty-five years after a streak of brutal mur...,15000000.0,1.730466e+08,AU,"Horror, Mystery, Thriller",01/13/2022,2022,tt11245972,movie,Scream,Scream,0,2022,\N,114,"Horror,Mystery,Thriller",6.3,169978
114,The Lost City,Reclusive author Loretta Sage writes about exo...,74000000.0,1.829220e+08,AU,"Action, Adventure, Comedy",03/30/2022,2022,tt13320622,movie,The Lost City,The Lost City,0,2022,\N,112,"Action,Adventure,Comedy",6.1,167659
115,The Lost City,Reclusive author Loretta Sage writes about exo...,9600000.0,5.256839e+06,AU,"Action, Adventure, Comedy",03/30/2022,2022,tt13320622,movie,The Lost City,The Lost City,0,2022,\N,112,"Action,Adventure,Comedy",6.1,167659
141,The Avengers,When an unexpected enemy emerges and threatens...,225000000.0,1.515100e+09,AU,"Science Fiction, Action, Adventure",04/25/2012,2012,tt0848228,movie,The Avengers,The Avengers,0,2012,\N,143,"Action,Sci-Fi",8.0,1492672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5077,The Birth of a Nation,"Two families, abolitionist Northerners the Sto...",10000000.0,1.689101e+07,US,"Drama, History, War",02/08/1915,1915,tt0004972,movie,The Birth of a Nation,The Birth of a Nation,0,1915,\N,195,"Drama,War",6.1,27076
5078,The Birth of a Nation,"Two families, abolitionist Northerners the Sto...",110000.0,1.100000e+07,US,"Drama, History, War",02/08/1915,1915,tt0004972,movie,The Birth of a Nation,The Birth of a Nation,0,1915,\N,195,"Drama,War",6.1,27076
5086,Pinocchio,Roberto Benigni adapts the classic children's ...,45000000.0,3.168181e+07,IT,"Comedy, Family, Fantasy",10/11/2002,2002,tt0255477,movie,Pinocchio,Pinocchio,0,2002,\N,108,"Comedy,Family,Fantasy",4.3,8746
5087,Pinocchio,Roberto Benigni adapts the classic children's ...,13200000.0,3.036038e+07,IT,"Comedy, Family, Fantasy",10/11/2002,2002,tt0255477,movie,Pinocchio,Pinocchio,0,2002,\N,108,"Comedy,Family,Fantasy",4.3,8746


In [10]:
"""Merge csv files."""
# Load and filter title basics
title_basics = pd.read_csv('assets/title.basics.csv')
title_basics_filtered = title_basics.query('titleType in ["movie", "tvSeries"]')

# Load and process IMDb movies
imdb_movies = pd.read_csv('assets/imdb_movies.csv', usecols=['orig_title', 'overview', 'budget_x', 'revenue', 'country', 'genre', 'date_x'])
imdb_movies['year_x'] = pd.to_datetime(imdb_movies['date_x'], errors='coerce').dt.year.astype(str)

# Merge IMDb movies with filtered title basics
merged_data = pd.merge(
    imdb_movies, title_basics_filtered,
    left_on=['orig_title', 'year_x'],
    right_on=['originalTitle', 'startYear'],
    how='left'
)

# Load title ratings and merge with existing data
title_ratings = pd.read_csv('assets/title.ratings.csv')
merged_data = pd.merge(
    merged_data, title_ratings,
    on='tconst',
    how='left'
)

# Remove duplicates based on 'tconst'
merged_data = merged_data.drop_duplicates(subset='tconst')

merged_data['year_x'] = pd.to_numeric(merged_data['year_x'], errors='coerce')


country_avg_rating = merged_data.groupby('country')['averageRating'].mean().reset_index()

display(country_avg_rating)



/var/folders/cf/g9jbbqjx6zq_3xw0tdrvfhx40000gn/T/ipykernel_63036/2461870797.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics = pd.read_csv('assets/title.basics.csv')


,country,averageRating
0,AR,6.166667
1,AU,6.415068
2,BE,5.633333
3,BR,4.966667
4,CA,5.730952
5,CH,5.950000
6,CL,4.925000
7,CN,6.033333
8,CO,4.800000
9,DE,5.616000


In [13]:
import pycountry

def iso2_to_country_name(iso2_code):
    try:
        # Convert ISO-2 to full country name using pycountry
        return pycountry.countries.get(alpha_2=iso2_code).name
    except AttributeError:
        return None  # Return None if the code is invalid or not found

In [14]:
merged_data['iso3_code'] = merged_data['country'].apply(iso2_to_country_name)

# Drop rows where conversion failed (invalid ISO-2 codes)
merged_data = merged_data.dropna(subset=['iso3_code'])

display(merged_data['iso3_code'].unique())

array(['Australia', 'United States', 'United Kingdom', 'Chile', 'Germany',
       'Spain', 'France', 'Mexico', 'Guatemala', 'Brazil', 'South Africa',
       'Netherlands', 'Finland', 'Indonesia', 'Denmark', 'Japan',
       'Ireland', 'Belgium', 'Argentina', 'Philippines', 'Türkiye',
       'Italy', 'Canada', 'Switzerland', 'China', 'Peru', 'Singapore',
       'Poland', 'India', 'Greece', 'Russian Federation',
       'Taiwan, Province of China', 'Malaysia', 'Mauritius', 'Norway',
       'Ukraine', 'Hong Kong', 'Sweden', 'Colombia'], dtype=object)